In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

In [6]:
def check_if_single_token(token):
    return len(tokenizer.tokenize(token)) == 1

task_name = "es_en"

task = tasks[task_name]

print(len(task))

task = {
    k:v for k,v in task.items() if check_if_single_token(k) and check_if_single_token(v)
}

print(len(task))

pairs = list(task.items())

763
346


In [7]:
from sprint.task_vector_utils import logprob_loss
from functools import partial

sep = 3978
pad = 0

def metric_fn(logits, resids, tokens):
    return logprob_loss(logits, tokens, sep=sep, pad_token=pad, n_first=2)

In [8]:
from micrlhf.llama import LlamaBlock, LlamaAttention
from micrlhf.utils.activation_manipulation import ActivationAddition, wrap_vector
from functools import partial
import jax.numpy as jnp
from penzai import pz
import jax

@partial(jax.jit, static_argnames=("metric", "batched"))
def run_with_add(additions_pre, additions_mid, tokens, metric, batched=False, llama=None):
    get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
            x
        ])
    )
    get_resids = get_resids.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda l, b: b.select().at_instances_of(pz.nn.Residual).apply_with_selected_index(lambda i, x: x if i == 0 else pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_mid_{l}"),
        x,
    ])))


    get_resids = get_resids.select().at_instances_of(LlamaAttention).apply_with_selected_index(lambda i, x: x.select().at_instances_of(pz.nn.Softmax).apply(lambda b: pz.nn.Sequential([
        b,
        pz.de.TellIntermediate.from_config(tag=f"attn_{i}"),
    ])))

    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: True)
    make_additions = get_resids.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
        pz.nn.Sequential([
            ActivationAddition(pz.nx.wrap(additions_pre[i], *(("batch",) if batched else ()), "seq", "embedding"), "all"),
            x
        ])
    )
    make_additions = make_additions.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda l, b: b.select().at_instances_of(pz.nn.Residual).apply_with_selected_index(lambda i, x: x if i == 0 else pz.nn.Sequential([
        ActivationAddition(pz.nx.wrap(additions_mid[l], *(("batch",) if batched else ()), "seq", "embedding"), "all"),
        x,
    ])))
    tokens_wrapped = pz.nx.wrap(tokens, "batch", "seq")
    logits, resids = make_additions(llama.inputs.from_basic_segments(tokens_wrapped))
    return metric(logits.unwrap("batch", "seq", "vocabulary"), resids, tokens), (logits, resids[::3], resids[1::3], resids[2::3])


@partial(jax.jit, static_argnames=("metric",))
def get_metric_resid_grad(tokens, llama=llama, metric=metric_fn):
    additions = [jnp.zeros(tokens.shape + (llama.config.hidden_size,)) for _ in range(llama.config.num_layers)]
    batched = tokens.ndim > 1
    (metric, (logits, resids_pre, qk, resids_mid)), (grad_pre, grad_mid) = jax.value_and_grad(run_with_add, argnums=(0, 1), has_aux=True)(additions, additions, tokens, metric, batched=batched, llama=llama)
    return (
        metric,
        [r.value.unwrap("batch", "seq", "embedding") for r in resids_pre],
        [r.value.unwrap("batch", "seq", "embedding") for r in resids_mid],
        [r.value.unwrap("batch", "kv_heads", "q_rep", "seq", "kv_seq") for r in qk],
        grad_pre,
        grad_mid
    )


In [9]:
batch_size = 8 
n_shot=20
max_seq_len = 128
seed = 10

In [37]:
prompt = "Follow the pattern:\n{}"

In [26]:
runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

In [27]:
from sprint.task_vector_utils import tokenized_to_inputs

train_tokens = runner.get_tokens(
    runner.train_pairs, tokenizer
)["input_ids"]

In [30]:
metric_value, resids_pre, resids_mid, qk, grad_pre, grad_mid = get_metric_resid_grad(train_tokens, llama=llama)

In [31]:
from micrlhf.utils.load_sae import get_nev_it_sae_suite

In [32]:
layer = 12

In [33]:
sae = get_nev_it_sae_suite(layer=layer)

In [34]:
def weights_to_resid(weights, sae):
    if "s_gate" in sae:
        weights = (weights > 0) * jax.nn.relu(weights * jax.nn.softplus(sae["s_gate"]) * sae.get("scaling_factor", 1.0) + sae["b_gate"])   
    else:
        weights = jax.nn.relu(weights)

    recon = jnp.einsum("fv,bsf->bsv", sae["W_dec"], weights)

    if "norm_factor" in sae:
        recon = recon / sae["norm_factor"]

    # recon = recon.astype('bfloat16')
    return recon

In [35]:
from micrlhf.utils.load_sae import sae_encode_gated

In [64]:
from tqdm.auto import tqdm

ie_attn = {}
ie_resid = {}
ie_transcoder = {}

ie_error_attn = {}
ie_error_resid = {}
ie_error_transcoder = {}

def sfc_simple(grad, resid, target, sae):
    pre_relu, post_relu, recon = sae_encode_gated(sae, resid)
    error = target - recon
    f = partial(weights_to_resid, sae=sae)

    sae_grad, = jax.vjp(f, post_relu)[1](grad,)
    indirect_effects = sae_grad * post_relu
    indirect_effects_error = grad * error
    return indirect_effects, indirect_effects_error

layers = list(range(6, 17))
for layer in tqdm(layers):
    r_pre, r_mid, g_mid = resids_pre[layer], resids_mid[layer], grad_mid[layer]
    sae = get_nev_it_sae_suite(layer=layer, label="attn_out")
    indirect_effects, indirect_effects_error = sfc_simple(g_mid, r_mid - r_pre, r_mid - r_pre, sae)
    # display((indirect_effects > 0).sum(-1))
    ie_attn[layer] = indirect_effects
    ie_error_attn[layer] = indirect_effects_error

# for layer, (r_pre, g_pre) in enumerate(zip(resids_pre, grad_pre)):
for layer in tqdm(layers):
    r_pre, g_pre = resids_pre[layer], grad_pre[layer + 1]
    sae = get_nev_it_sae_suite(layer=layer)
    indirect_effects, indirect_effects_error = sfc_simple(g_pre, r_pre, r_pre, sae)
    # display((indirect_effects != 0).sum(-1))
    ie_resid[layer] = indirect_effects
    ie_error_resid[layer] = indirect_effects_error

for layer in tqdm(layers[:-1]):
    r_mid, r_pre, g_pre = resids_mid[layer], resids_pre[layer + 1], grad_pre[layer + 1]
    sae = get_nev_it_sae_suite(layer=layer, label="transcoder")
    indirect_effects, indirect_effects_error = sfc_simple(g_pre, r_mid, r_pre, sae)
    # display((indirect_effects != 0).sum(-1))
    ie_transcoder[layer] = indirect_effects
    ie_error_transcoder[layer] = indirect_effects_error

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [53]:
prompt_length = len(tokenizer.tokenize(prompt))
periods = ["input", "arrow", "output", "newline"]
masks = {
    "prompt": jnp.zeros_like(train_tokens).at[:, :prompt_length].set(1).astype(bool),
    **{
        period: jnp.zeros_like(train_tokens).at[:, prompt_length+i::len(periods)].set(1).astype(bool) * (train_tokens != pad) for i, period in enumerate(periods)
    }
}

In [59]:
def compute_avg_node_effect(ie, top_k=64):
    masked = {}
    for mask_name, mask in masks.items():
        mask = mask[..., None]
        masked[mask_name] = ((mask * jnp.abs(ie)).sum(1) / mask.sum(1)).mean(0)
    masked = {k: jax.lax.top_k(v, k=top_k) for k,v in masked.items()}
    return masked
def compute_all_avg_node_effects(ies):
    return {k: compute_avg_node_effect(v) for k,v in ies.items()}
compute_all_avg_node_effects(ie_resid)

{'prompt': [# jax.Array float32(64,) ≈8.9e-06 ±1.5e-05 [≥1.3e-06, ≤7.7e-05] nonzero:64
    Array([7.71635023e-05, 7.16703726e-05, 4.67821083e-05, 3.87677028e-05,
           3.71553615e-05, 2.33888622e-05, 1.80917632e-05, 1.16701285e-05,
           1.13662600e-05, 1.11477539e-05, 9.89903037e-06, 9.62063677e-06,
           9.42581573e-06, 9.36959896e-06, 8.78927676e-06, 8.71421526e-06,
           8.24398103e-06, 7.48182993e-06, 7.47837021e-06, 7.40456153e-06,
           7.25703421e-06, 7.02204761e-06, 6.44178181e-06, 6.28425960e-06,
           6.21455001e-06, 5.48434173e-06, 5.45412695e-06, 5.30116813e-06,
           5.07138520e-06, 4.43589488e-06, 4.16759940e-06, 4.00389763e-06,
           3.94163681e-06, 3.67492885e-06, 3.55562338e-06, 3.29629302e-06,
           3.24763005e-06, 3.06661241e-06, 2.87246303e-06, 2.80252743e-06,
           2.55035707e-06, 2.50999051e-06, 2.43364116e-06, 2.38233497e-06,
           2.28199610e-06, 2.22419317e-06, 2.18208083e-06, 2.16460649e-06,
           2.15639920e-06, 1.98139332e-06, 1.93602841e-06, 1.89844343e-06,
           1.83689872e-06, 1.78019843e-06, 1.74903255e-06, 1.66514553e-06,
           1.65828408e-06, 1.59510114e-06, 1.54371685e-06, 1.45570584e-06,
           1.37404118e-06, 1.35749076e-06, 1.35518292e-06, 1.32207526e-06],      dtype=float32)
  ,
  # jax.Array int32(64,) [≥288, ≤31_816] nonzero:64
    Array([26908, 24884, 31316, 27225, 23001, 24176, 11898, 28246, 16564,
           28244, 16682, 12601, 19813,  3478,  9186, 24799, 30404,  4825,
           26140,  3097, 31606,  2350, 18879, 16370, 29211, 24631,  5623,
           31427,  8825, 27699, 20709, 22039, 26734, 14727, 31816, 21308,
            9914, 24936, 10850,  1379, 25595, 30819,  7015,   288,  9896,
           20285, 28462,   716, 23106, 30735, 14780, 24114, 12653, 12272,
            4265, 29969, 23606, 18357, 29825, 13508, 12956,  7377, 20898,
            2064], dtype=int32)
  ],
 'input': [# jax.Array float32(64,) ≈5.1e-05 ±8.7e-05 [≥1.2e-05, ≤0.00066] nonzero:64
    Array([6.6398986e-04, 1.8403497e-04, 1.5831170e-04, 1.4810490e-04,
           1.3826501e-04, 1.0589224e-04, 9.3690964e-05, 9.3001348e-05,
           9.2378315e-05, 9.2126080e-05, 8.6977358e-05, 8.3581617e-05,
           8.3282153e-05, 7.2516668e-05, 6.6209162e-05, 5.4997825e-05,
           5.0765153e-05, 4.9010043e-05, 4.5527922e-05, 3.6774651e-05,
           3.2010721e-05, 3.1732339e-05, 3.1728719e-05, 2.7966780e-05,
           2.7596101e-05, 2.5726422e-05, 2.5674008e-05, 2.4935638e-05,
           2.4226525e-05, 2.3629516e-05, 2.3120703e-05, 2.2974320e-05,
           2.2772056e-05, 2.2347111e-05, 2.1526106e-05, 2.1111711e-05,
           2.0494193e-05, 2.0480000e-05, 1.9620231e-05, 1.8988265e-05,
           1.8579287e-05, 1.8390436e-05, 1.7310269e-05, 1.7157152e-05,
           1.7039085e-05, 1.7038561e-05, 1.6636446e-05, 1.5940561e-05,
           1.5331789e-05, 1.4919287e-05, 1.4417552e-05, 1.4362228e-05,
           1.4350079e-05, 1.4147059e-05, 1.4139234e-05, 1.3929453e-05,
           1.3770119e-05, 1.3462327e-05, 1.3120892e-05, 1.2330558e-05,
           1.2226375e-05, 1.2214146e-05, 1.2195540e-05, 1.2063870e-05],      dtype=float32)
  ,
  # jax.Array int32(64,) [≥46, ≤31_800] nonzero:64
    Array([ 9049, 19293, 16969, 28738, 29964,  1388, 21942,  4338, 20768,
           12941, 28488, 30927,  8749, 24348,    46,  4641, 27786,  7444,
           11025,  5334, 13025, 10415, 31800, 13426, 26302, 23355,  4675,
           22990,  6402,  4622,  7552,  3806, 12359,  9765, 25433, 24130,
            2896, 12688, 17355, 24343, 16154, 28648,  2055, 18924, 13540,
             773, 22500, 11374, 18010,  6322, 17541, 30153, 26287,  3388,
           28175,  4502, 21344,  3365,  7804, 28941, 23690,  2697,  9831,
           25003], dtype=int32)
  ],
 'arrow': [# jax.Array float32(64,) ≈7.6e-05 ±7e-05 [≥1.8e-05, ≤0.00042] nonzero:64
    Array([4.17096162e-04, 2.71777128e-04, 2.44092604e-04, 1.91443847e-04,
           1.79518087e-04, 1.76569563e-04, 1.70449086e-04, 1.68896557

In [ ]:
def mid_to_pre_grad(layer, resid):
    block = llama.select().